# Live Code 3 Phase 0

Nama: Sonny Riadi

Batch: FTDS-009

### Import Library

In [1]:
import pandas as pd
from scipy import stats

### Data Loading

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/fahmimnalfrzki/Dataset/main/purchase_data.txt?token=AEZDEHRU5YLQEXGDDLCL6CLA2KZTI',delimiter=';')

In [3]:
df.head(10)

,user_id,timestamp,group,landing_page,purchased
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
7,719014,2017-01-17 01:48:29.539573,control,old_page,0
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


Keterangan Dataset:

    user_id = id pelanggan
    timestamp = Waktu ketika pelanggan mengakses laman web
    group = grup kontrol dan treatment
    landing_page = landing page yang diakses oleh pelanggan (laman lama atau laman baru)
    purchased = 0 : tidak ada pembelian, and 1: ada pembelian


In [4]:
df.shape

(294478, 5)

### Data Cleaning

In [5]:
df.head(10)

,user_id,timestamp,group,landing_page,purchased
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
7,719014,2017-01-17 01:48:29.539573,control,old_page,0
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


##### Missing Values

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   purchased     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [7]:
for col in df.columns:
    missing_value = df[col].isnull().value_counts()
    print(missing_value)

False    294478
Name: user_id, dtype: int64
False    294478
Name: timestamp, dtype: int64
False    294478
Name: group, dtype: int64
False    294478
Name: landing_page, dtype: int64
False    294478
Name: purchased, dtype: int64


dapat diketahui tidak ada missing values untuk semua column

##### Invalid Values

In [8]:
df['purchased'].unique()

array([0, 1])

dapat diketahui tidak ada invalid values untuk column purchased

In [9]:
df['landing_page'].unique()

array(['old_page', 'new_page'], dtype=object)

dapat diketahui tidak ada invalid values untuk column landing page

##### Duplicated Values

In [10]:
df['user_id'].duplicated().value_counts()

False    290584
True       3894
Name: user_id, dtype: int64

diketahui terdapat duplicated values untuk user ID sebanyak 3894 values, sehingga perlu kita hapus.

In [11]:
df = df.drop_duplicates(subset=['user_id'])

# Problem

Kamu adalah seorang data scientist di salah satu perusahaan e-commerce. Saat ini, kamu terlibat dalam proyek bersama dengan tim produk dan tim UI/UX untuk merubah tampilan `landing page`

### Data Preparation

In [12]:
df.head(10)

,user_id,timestamp,group,landing_page,purchased
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
7,719014,2017-01-17 01:48:29.539573,control,old_page,0
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


dikarenakan kita akan menguji A/B Testing kita memiliki 2 variabel yaitu `old_page` & `new_page`

##### Data preparation for variabel old page

In [13]:
df_old_page = df[
    (df['landing_page']=='old_page')
][['purchased']]
df_old_page

,purchased
0,0
1,0
4,1
5,0
7,0
...,...
294471,0
294473,0
294474,0
294475,0


##### Data preparation for variabel new page

In [14]:
df_new_page = df[
    (df['landing_page']=='new_page')
][['purchased']]
df_new_page

,purchased
2,0
3,0
6,1
8,1
9,1
...,...
294462,0
294465,0
294468,0
294472,0



### Central Tendency

In [15]:
print('Mean :')
print('Old page :' +str(round(df_old_page.purchased.mean(),4)))
print('New page :' +str(round(df_new_page.purchased.mean(),4)))
print('')
print('Median :')
print('Old page :' +str(df_old_page.purchased.median()))
print('New page :' +str(df_new_page.purchased.median()))
print('')
print('Modus :')
print('Old page :' +str(df_old_page.purchased.mode()[0]))
print('New page :' +str(df_new_page.purchased.mode()[0]))

Mean :
Old page :0.1204
New page :0.1187

Median :
Old page :0.0
New page :0.0

Modus :
Old page :0
New page :0


**2a. Berdasarkan hasil perhitungan central tendency, data cenderung berpusat ke purchased atau non purchased? apa kesimpulanmu?**

Berdasarkan hasil perhitungan central tendency untuk old page dan new page, dapat disimpulkan dari kedua data tersebut cenderung berpusat ke `non-purchased` hal ini dapat diketahui dari nilai Modus atau nilai yang paling sering muncul yaitu 0 (non-purchased) dan rata-rata yang mendekati nilai 0 (non-purchased)

## A/B Testing

#### Purchased Rate

In [16]:
print('Old Page Purchased Rate :',df_old_page.sum()/df_old_page.count())
print('Old Page Purchased Rate :',df_new_page.sum()/df_new_page.count())

Old Page Purchased Rate : purchased    0.120395
dtype: float64
Old Page Purchased Rate : purchased    0.118745
dtype: float64


berdasarkan perhitungan purchased rate untuk kedua jenis landing page dapat dilihat jika kedua jenis tersebut memeiliki purchased rate yang sama, untuk lebih yakin maka kita akan melakukan A/B Testing



**2b. Definisikan H0 dan H1 dari hipotesis testingnya!**

H0: μ puchased old page = μ purchased new page

H1: μ puchased old page != μ purchased new page

**2c. Jenis hipotesis apa yang akan kamu gunakan?**

Paired Test A/B Testing

In [17]:
t_stat,p_val = stats.ttest_rel(df_old_page.purchased.sample(140000),df_new_page.purchased.sample(140000)) # we take 140000 sample for each to make it the same size as population
print('P-value:',p_val)
print('T-stat:',t_stat)

P-value: 0.14984883508625924
T-stat: 1.4400735232654116


**2d. Apa kesimpulanmu terhadap hasil A/B testing tsb?**

berdasarkan hasil uji hipotesis Paired Test A/B Testing, dapat dilihat nilai P-value lebih besar dari 0.05 sehingga dapat disimpulkan bahwa tidak ada perbedaan `purchased`  yang signifikan diantara `old page` dan `new page`, sehingga perusahaan dapat merubah tampilan landing page.